In [3]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
news = pd.read_csv("train.csv")

In [5]:
news.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
news.shape

(20800, 5)

In [7]:
# counting the number of null values
news.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
# converting null values to empty strings
news = news.fillna(' ')

In [9]:
news.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [10]:
# merging required columns
news['Content'] = news['author'] + " " + news['title']

In [11]:
news.head()

,id,title,author,text,label,Content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [12]:
# stemming the data
ps = PorterStemmer()

In [13]:
def stemming(data):
    stemmed_data = re.sub('[^a-zA-Z]', " ", data)
    stemmed_data = stemmed_data.lower()
    stemmed_data = stemmed_data.split()
    stemmed_data = [ ps.stem(word) for word in stemmed_data if word not in stopwords.words('english')]
    stemmed_data = ' '.join(stemmed_data)
    return stemmed_data

In [14]:
news['Content'] = news['Content'].apply(stemming)

In [15]:
news['Content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: Content, Length: 20800, dtype: object

In [16]:
# separating the feature and label of training data
x = news['Content'].values
y = news['label'].values

In [17]:
# convert text into numerical values
vector = TfidfVectorizer()
vector.fit(x)
x = vector.transform(x)

## Splitting the data

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, stratify = y, random_state=1)

## Training the model using Logistic Regression

In [19]:
model = LogisticRegression()

In [20]:
model.fit(x_train,y_train)

LogisticRegression()

In [21]:
y_tr_pred = model.predict(x_train)

In [22]:
accuracy_train = accuracy_score(y_tr_pred, y_train)

In [23]:
print("Accuracy score of the training data: ", accuracy_train)

Accuracy score of the training data:  0.9868389423076923


In [24]:
y_te_pred = model.predict(x_test)

In [25]:
accuracy_test = accuracy_score(y_te_pred, y_test)

In [26]:
print("Accuracy score of the testing data: ", accuracy_test)

Accuracy score of the testing data:  0.9766826923076923


In [27]:
# prediction system

input_data = x_test[10]
prediction = model.predict(input_data)
if prediction[0] == 1:
    print("Fake News")
else:
    print("Real News")

Real News


In [28]:
import pickle

In [29]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, "wb"))

In [30]:
loaded_model = pickle.load(open('trained_model.sav','rb'))

In [31]:
# prediction system

input_data = x_test[10]
prediction = loaded_model.predict(input_data)
if prediction[0] == 1:
    print("Fake News")
else:
    print("Real News")

Real News
